In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import (Conv2D, Dropout, MaxPool2D, Flatten, Dense, BatchNormalization,)
from keras.constraints import MaxNorm
from keras.datasets.cifar10 import load_data

 #load data CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = load_data()

# rescale image
x_train_scaled = x_train / 255.0
x_test_scaled = x_test / 255.0

num_class = 10

# printar pra ver se tá tudo certo
print('Train: X=%s, y=%s' % (x_train.shape, y_train.shape))
print('Test: X=%s, y=%s' % (x_test.shape, y_test.shape))


Train: X=(50000, 32, 32, 3), y=(50000, 1)
Test: X=(10000, 32, 32, 3), y=(10000, 1)


In [2]:
#define model
model = Sequential([
    Conv2D(32, (2,2), input_shape=(32, 32, 3), padding="same", activation="relu", kernel_constraint=MaxNorm(3)),

    BatchNormalization(),

    Conv2D(32, (2,2), input_shape=(32, 32, 3), padding="same", activation="relu", kernel_constraint=MaxNorm(3)),

    BatchNormalization(),

    MaxPool2D((2, 2), strides=2, padding="same"),

    Dropout(0.2),

    Conv2D(64, (2,2), input_shape=(32, 32, 3), padding="same", activation="relu", kernel_constraint=MaxNorm(3)),

    BatchNormalization(),

    Conv2D(64, (2,2), input_shape=(32, 32, 3), padding="same", activation="relu", kernel_constraint=MaxNorm(3)),

    BatchNormalization(),

    MaxPool2D((2,2), strides=2, padding="same"),

    Dropout(0.25),

    Conv2D(128, (2,2), input_shape=(32, 32, 3), padding="same", activation="relu", kernel_constraint=MaxNorm(3)),

    BatchNormalization(),

    Conv2D(128, (2,2), input_shape=(32, 32, 3), padding="same", activation="relu", kernel_constraint=MaxNorm(3)),

    BatchNormalization(),

    MaxPool2D((2, 2), strides=2, padding="same"),

    Dropout(0.5),

    Flatten(),

    Dense(128, activation="relu", kernel_constraint=MaxNorm(3)),

    Dropout(0.5),

    Dense(num_class, activation="softmax")
])

# summarize model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        416       
                                                                 
 batch_normalization (Batch  (None, 32, 32, 32)        128       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        4128      
                                                                 
 batch_normalization_1 (Bat  (None, 32, 32, 32)        128       
 chNormalization)                                                
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 16, 32)        0         
 D)                                                              
                                                        

In [ ]:
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",
              metrics="sparse_categorical_accuracy")

model.fit(x_train_scaled, y_train, validation_data=(x_test_scaled, y_test), epochs=25, batch_size=32, workers=12)


Epoch 1/25
1563/1563 [==============================] - 27s 11ms/step - loss: 1.8469 - sparse_categorical_accuracy: 0.3137 - val_loss: 1.4557 - val_sparse_categorical_accuracy: 0.4585
Epoch 2/25
1563/1563 [==============================] - 16s 10ms/step - loss: 1.4736 - sparse_categorical_accuracy: 0.4561 - val_loss: 1.1399 - val_sparse_categorical_accuracy: 0.5803
Epoch 3/25
1563/1563 [==============================] - 16s 10ms/step - loss: 1.2965 - sparse_categorical_accuracy: 0.5384 - val_loss: 1.1774 - val_sparse_categorical_accuracy: 0.5937
Epoch 4/25
1563/1563 [==============================] - 16s 10ms/step - loss: 1.1780 - sparse_categorical_accuracy: 0.5864 - val_loss: 0.9382 - val_sparse_categorical_accuracy: 0.6694
Epoch 5/25
1563/1563 [==============================] - 17s 11ms/step - loss: 1.0872 - sparse_categorical_accuracy: 0.6189 - val_loss: 0.9679 - val_sparse_categorical_accuracy: 0.6650
Epoch 6/25
1563/1563 [==============================] - 17s 11ms/step - loss: 1.

In [ ]:

#you can extract the output of each layer using an extractor model.
# Extract output from each layer
extractor = tf.keras.Model(inputs=model.inputs,
                           outputs=[layer.output for layer in model.layers])
features = extractor(np.expand_dims(x_train[7], 0))

# Show the 32 feature maps from the first layer
l0_features = features[0].numpy()[0]

fig, ax = plt.subplots(4, 8, sharex=True, sharey=True, figsize=(16,8))
for i in range(0, 32):
    row, col = i//8, i%8
    ax[row][col].imshow(l0_features[..., i])

plt.show()

In [ ]:

# Show the 32 feature maps from the fourth layer
l4_features = features[4].numpy()[0]

fig, ax = plt.subplots(4, 8, sharex=True, sharey=True, figsize=(16,8))
for i in range(0, 32):
    row, col = i//8, i%8
    ax[row][col].imshow(l4_features[..., i])

plt.show()